In [1643]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import numpy as np

The Nim and Nimply classes

In [1644]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [1645]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        bool = False
        if sum(self._rows) > 0:
            bool = True
        return bool

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> np.array:
        return np.array(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

Random strategy

In [1646]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [1647]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)
    
    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

A hard-coded strategy

In [1648]:
def make_strategy1() -> Callable:
    def hardcoded(state: Nim) -> Nimply:
        data = cook_status(state)

        if (state.rows.sum() % 2 == 0 and state.rows[data["longest_row"]] > 1):
            ply = Nimply(data["longest_row"], 2)
        else:
            ply = Nimply(data["longest_row"], 1)

        return ply

    return hardcoded

Another hard-coded strategy

In [1649]:
def make_strategy2() -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if (data["active_rows_number"] % 2 == 0 and state.rows[data["longest_row"]] > 2):
            ply = Nimply(data["longest_row"], state.rows[data["longest_row"]] - 1)
        else:
            ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
            
        return ply

    return evolvable

An evolvable strategy

In [1650]:
def make_strategy3(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            if (data["active_rows_number"] % 2 == 0 and state.rows[data["longest_row"]] > 2):
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]] - 1)
            else:
                ply = Nimply(data["longest_row"], state.rows[data["longest_row"]])
        else:
            if (data["active_rows_number"] % 2 == 0 and state.rows[data["shortest_row"]] > 2):
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]] - 1)
            else:
                ply = Nimply(data["shortest_row"], state.rows[data["shortest_row"]])
                
        return ply

    return evolvable

Evaluation

In [1651]:
NUM_MATCHES = 100
NIM_SIZE = 10

#evaluation with pure_random as the second player
def evaluate1(strategy: Callable) -> float:
    opponent = (strategy, pure_random)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES


#evaluation with pure_random as the first player
def evaluate2(strategy: Callable) -> float:
    opponent = (pure_random, strategy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 0:
            won += 1
    return won / NUM_MATCHES    

In [1652]:
logging.getLogger().setLevel(logging.DEBUG)

#evaluating the first hard-coded strategy
logging.debug(f"first hard-coded strategy vs. pure_random: {evaluate1(make_strategy1())}")
logging.debug(f"pure_random vs. first hard-coded strategy: {evaluate2(make_strategy1())}")
#evaluating the second hard-coded strategy
logging.debug(f"second hard-coded strategy vs. pure_random: {evaluate1(make_strategy2())}")
logging.debug(f"pure_random vs. second hard-coded strategy: {evaluate2(make_strategy2())}")

DEBUG:root:first hard-coded strategy vs. pure_random: 0.9
DEBUG:root:pure_random vs. first hard-coded strategy: 0.86
DEBUG:root:second hard-coded strategy vs. pure_random: 0.85
DEBUG:root:pure_random vs. second hard-coded strategy: 0.91


In [1672]:
NUM_GENERATIONS = 30

p1 = 0.5
oldEval = 0
for g in range(NUM_GENERATIONS):
    newEval = evaluate1(make_strategy3({"p": p1}))
    if newEval > oldEval:
        p1 += 0.1
        if p1 > 1:
            p1 = 1
        oldEval = newEval
    else:
        p1 -= 0.1
        if p1 < 0:
            p1 = 0
        oldEval = newEval
logging.debug(f"probability of choosing the longest row when playing as the first player: {p1}")

p2 = 0.5
oldEval = 0
for g in range(NUM_GENERATIONS):
    newEval = evaluate2(make_strategy3({"p": p2}))
    if newEval > oldEval:
        p2 += 0.1
        if p2 > 1:
            p2 = 1
        oldEval = newEval
    else:
        p2 -= 0.1
        if p2 < 0:
            p2 = 0
        oldEval = newEval
logging.debug(f"probability of choosing the longest row when playing as the second player: {p2}")

DEBUG:root:probability of choosing the longest row when playing as the first player: 0.30000000000000004
DEBUG:root:probability of choosing the longest row when playing as the second player: 0.30000000000000004


In [1673]:
#evaluating the evolvable strategy
e1 = evaluate1(make_strategy3({"p": 0.3}))
logging.debug(f"second hard-coded strategy vs. pure_random: {e1}")
e2 = evaluate2(make_strategy3({"p": 0.3}))
logging.debug(f"pure_random vs. second hard-coded strategy: {e2}")

DEBUG:root:second hard-coded strategy vs. pure_random: 0.86
DEBUG:root:pure_random vs. second hard-coded strategy: 0.82


Oversimplified match

In [ ]:
strategy = (make_strategy1(), pure_random)
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 20>
DEBUG:root:status: After player 1 -> <1 1 5 7 9 11 13 15 17 19 20>
DEBUG:root:status: After player 0 -> <1 1 5 7 9 11 13 15 17 19 18>
DEBUG:root:status: After player 1 -> <1 1 0 7 9 11 13 15 17 19 18>
DEBUG:root:status: After player 0 -> <1 1 0 7 9 11 13 15 17 18 18>
DEBUG:root:status: After player 1 -> <1 1 0 7 9 11 5 15 17 18 18>
DEBUG:root:status: After player 0 -> <1 1 0 7 9 11 5 15 17 16 18>
DEBUG:root:status: After player 1 -> <1 1 0 7 9 11 5 2 17 16 18>
DEBUG:root:status: After player 0 -> <1 1 0 7 9 11 5 2 17 16 17>
DEBUG:root:status: After player 1 -> <1 1 0 7 2 11 5 2 17 16 17>
DEBUG:root:status: After player 0 -> <1 1 0 7 2 11 5 2 16 16 17>
DEBUG:root:status: After player 1 -> <1 1 0 7 0 11 5 2 16 16 17>
DEBUG:root:status: After player 0 -> <1 1 0 7 0 11 5 2 16 16 15>
DEBUG:root:status: After player 1 -> <1 0 0 7 0 11 5 2 16 16 15>
DEBUG:root:

In [ ]:
strategy = (pure_random, make_strategy1())
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <0 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 1 -> <0 3 5 7 9 11 13 15 17 19 19>
DEBUG:root:status: After player 0 -> <0 3 5 7 9 11 0 15 17 19 19>
DEBUG:root:status: After player 1 -> <0 3 5 7 9 11 0 15 17 18 19>
DEBUG:root:status: After player 0 -> <0 3 5 1 9 11 0 15 17 18 19>
DEBUG:root:status: After player 1 -> <0 3 5 1 9 11 0 15 17 18 17>
DEBUG:root:status: After player 0 -> <0 3 5 1 6 11 0 15 17 18 17>
DEBUG:root:status: After player 1 -> <0 3 5 1 6 11 0 15 17 17 17>
DEBUG:root:status: After player 0 -> <0 3 0 1 6 11 0 15 17 17 17>
DEBUG:root:status: After player 1 -> <0 3 0 1 6 11 0 15 16 17 17>
DEBUG:root:status: After player 0 -> <0 0 0 1 6 11 0 15 16 17 17>
DEBUG:root:status: After player 1 -> <0 0 0 1 6 11 0 15 16 16 17>
DEBUG:root:status: After player 0 -> <0 0 0 1 6 9 0 15 16 16 17>
DEBUG:root:status: After player 1 -> <0 0 0 1 6 9 0 15 16 16 15>
DEBUG:roo

In [ ]:
strategy = (make_strategy2(), pure_random)
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 0>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 1 15 17 19 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 1 15 17 1 0>
DEBUG:root:status: After player 1 -> <1 2 5 7 9 11 1 15 17 1 0>
DEBUG:root:status: After player 0 -> <1 2 5 7 9 11 1 15 1 1 0>
DEBUG:root:status: After player 1 -> <1 2 5 7 9 2 1 15 1 1 0>
DEBUG:root:status: After player 0 -> <1 2 5 7 9 2 1 1 1 1 0>
DEBUG:root:status: After player 1 -> <1 2 5 7 3 2 1 1 1 1 0>
DEBUG:root:status: After player 0 -> <1 2 5 1 3 2 1 1 1 1 0>
DEBUG:root:status: After player 1 -> <1 2 5 1 3 1 1 1 1 1 0>
DEBUG:root:status: After player 0 -> <1 2 1 1 3 1 1 1 1 1 0>
DEBUG:root:status: After player 1 -> <1 2 0 1 3 1 1 1 1 1 0>
DEBUG:root:status: After player 0 -> <1 2 0 1 0 1 1 1 1 1 0>
DEBUG:root:status: After player 1 -> <1 2 0 1 0 1 1 1 1 0 0>
DEBUG:root:status: After player 0 -> <1 0 0 1 0 1 1 1 1 0 0>


In [ ]:
strategy = (pure_random, make_strategy2())
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 10 21>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 13 15 17 10 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 11 15 17 10 0>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 11 15 1 10 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 7 15 1 10 0>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 7 1 1 10 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 7 0 1 10 0>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 0 7 0 1 10 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 6 0 7 0 1 10 0>
DEBUG:root:status: After player 1 -> <1 3 5 7 6 0 7 0 1 1 0>
DEBUG:root:status: After player 0 -> <1 3 5 7 6 0 7 0 0 1 0>
DEBUG:root:status: After player 1 -> <1 3 5 0 6 0 7 0 0 1 0>
DEBUG:root:status: After player 0 -> <1 3 3 0 6 0 7 0 0 1 0>
DEBUG:root:status: After player 1 -> <1 3 3 0 6 0 1 0 0 1 0>
DEBUG:root:status: After player 0 -> <1 3 3 0 6 0 

In [1675]:
strategy = (make_strategy3({"p": 0.3}), pure_random)
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 0>
DEBUG:root:status: After player 1 -> <1 3 5 1 9 11 13 15 17 19 0>
DEBUG:root:status: After player 0 -> <0 3 5 1 9 11 13 15 17 19 0>
DEBUG:root:status: After player 1 -> <0 3 4 1 9 11 13 15 17 19 0>
DEBUG:root:status: After player 0 -> <0 3 4 0 9 11 13 15 17 19 0>
DEBUG:root:status: After player 1 -> <0 3 4 0 9 11 5 15 17 19 0>
DEBUG:root:status: After player 0 -> <0 1 4 0 9 11 5 15 17 19 0>
DEBUG:root:status: After player 1 -> <0 1 4 0 9 11 5 15 10 19 0>
DEBUG:root:status: After player 0 -> <0 0 4 0 9 11 5 15 10 19 0>
DEBUG:root:status: After player 1 -> <0 0 4 0 9 11 5 15 10 8 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 9 11 5 15 10 8 0>
DEBUG:root:status: After player 1 -> <0 0 0 0 9 9 5 15 10 8 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 9 9 1 15 10 8 0>
DEBUG:root:status: After player 1 -> <0 0 0 0 9 9 0 15 10 8 0>
DEBUG:root:status: After p

In [1676]:
strategy = (pure_random, make_strategy3({"p": 0.3}))
nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 3>
DEBUG:root:status: After player 1 -> <0 3 5 7 9 11 13 15 17 19 3>
DEBUG:root:status: After player 0 -> <0 3 5 7 9 11 13 15 17 19 0>
DEBUG:root:status: After player 1 -> <0 3 5 7 9 11 13 15 17 0 0>
DEBUG:root:status: After player 0 -> <0 3 5 7 2 11 13 15 17 0 0>
DEBUG:root:status: After player 1 -> <0 3 5 7 2 11 13 15 1 0 0>
DEBUG:root:status: After player 0 -> <0 3 5 7 1 11 13 15 1 0 0>
DEBUG:root:status: After player 1 -> <0 3 5 7 1 11 13 1 1 0 0>
DEBUG:root:status: After player 0 -> <0 3 5 7 1 11 7 1 1 0 0>
DEBUG:root:status: After player 1 -> <0 3 5 7 0 11 7 1 1 0 0>
DEBUG:root:status: After player 0 -> <0 3 5 7 0 9 7 1 1 0 0>
DEBUG:root:status: After player 1 -> <0 3 5 7 0 0 7 1 1 0 0>
DEBUG:root:status: After player 0 -> <0 3 4 7 0 0 7 1 1 0 0>
DEBUG:root:status: After player 1 -> <0 3 4 7 0 0 7 0 1 0 0>
DEBUG:root:status: After player 0 -> <0 3 0 7 